In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [2]:
game_df = pd.read_csv("data/team_data.csv")
df = pd.read_csv("../data fetch/data/batting_player_2024.csv")

In [3]:
print(game_df.shape)
print(df.shape)

(4938, 10)
(51554, 28)


In [4]:
# Singles
df['singles'] = df['hits'] - df['doubles'] - df['triples'] - df['homeRuns']

# Batting Average
df['batting_avg'] = df['hits'] / df['atBats'].replace(0, np.nan)

# Slugging Percentage
df['slugging_pct'] = (df['singles'] + 2*df['doubles'] + 3*df['triples'] + 4*df['homeRuns']) / df['atBats'].replace(0, np.nan)

# On-Base Percentage
df['OBP'] = (df['hits'] + df['baseOnBalls'] + df['hitByPitch']) / \
                 (df['atBats'] + df['baseOnBalls'] + df['hitByPitch'] + df['sacFlies']).replace(0, np.nan)

# OPS
df['OPS'] = df['OBP'] + df['slugging_pct']

# Strikeout Rate
df['K_rate'] = df['strikeOuts'] / df['plateAppearances'].replace(0, np.nan)

# Walk Rate
df['BB_rate'] = df['baseOnBalls'] / df['plateAppearances'].replace(0, np.nan)

# Productive Outs
df['productive_outs'] = df['sacBunts'] + df['sacFlies']

### aggregated

In [5]:
# Group by gamePk + team_id
df = df.groupby(['gamePk', 'team_id']).agg({
    'batting_avg': 'mean',
    'OPS': 'mean',
    'K_rate': 'mean',
    'BB_rate': 'mean',
    'productive_outs': 'sum',   # counting stats sum
    'homeRuns': 'sum',
    'hits': 'sum',
    'rbi': 'sum',
    'doubles': 'sum',
    'triples': 'sum',
    'singles': 'sum'
}).reset_index()

# -------------------------
# Rename columns like player-level
# -------------------------

df.rename(columns={
    'batting_avg': 'batting_player_avg',
    'OPS': 'batting_player_OPS',
    'K_rate': 'batting_player_K_rate',
    'BB_rate': 'batting_player_BB_rate',
    'productive_outs': 'batting_player_productive_outs',
    'homeRuns': 'batting_player_homeRuns',
    'hits': 'batting_player_hits',
    'rbi': 'batting_player_rbi',
    'doubles': 'batting_player_doubles',
    'triples': 'batting_player_triples',
    'singles': 'batting_player_singles'
}, inplace=True)


In [6]:
# Merge batting_team ka team_side into batting_player
df = df.merge(
    game_df[['gamePk', 'team_id', 'team_side']],
    on=['gamePk', 'team_id'],
    how='left'
)

### wide format

In [7]:
# Split into home and away
batting_home = (
    df[df['team_side']=='home']
    .drop(columns=['team_side'])
    .add_prefix('home_')
    .rename(columns={'home_gamePk':'gamePk'})
)

batting_away = (
    df[df['team_side']=='away']
    .drop(columns=['team_side'])
    .add_prefix('away_')
    .rename(columns={'away_gamePk':'gamePk'})
)

# Merge home and away into wide format
df = pd.merge(batting_home, batting_away, on='gamePk', how='inner')

In [8]:
df.shape

(2550, 25)

In [9]:
df.to_csv("data/batting_player_clean.csv", index = False)